In [1]:
import os
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain_utils

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

### EVENT OCCURRENEC CACHE

In [4]:
container = {}

### CONFIG PARAMS

In [5]:
history_length = 5

In [6]:
error_margin = 0.2

### TRACKING FUNC

In [8]:
try:
    print('STARTED LISTENING...\n')

    # OPEN THE FILE
    file = open(settings['log_path'], 'r')

    # FIND THE TAILEND OF THE FILE
    file_stats = os.stat(settings['log_path'])
    file_size = file_stats[6]

    # GOTO THE LAST ROW
    file.seek(file_size)

    # EVENT LOOP
    while True:

        # READ THE LAST LINE
        where = file.tell()
        line = file.readline()

        # IF THE LINE ISNT EMPTY
        if line:

            # PARSE THE COMPLETE OBJECT
            parsed = utils.parse_line(line)

            # SEND HASH & TIMESTAMP FOR VERIFICATION
            hash_id = parsed['hash']
            timestamp = parsed['timestamp']

            # IF THE EVENT HAS OCCURRED BEFORE
            if hash_id in container:

                # CALCULATE TIMESTAMP DELTA BETWEEN CURRENT AND PREVIOUS
                delta = timestamp - container[hash_id]['previous']

                # SET PREVIOUS AS CURRENT
                container[hash_id]['timestamp'] = timestamp

                # IF THE HISTORY ISNT EMPTY
                if len(container[hash_id]['history']) > 0:

                    # AVERAGE HISTORY DISTANCE
                    avg = sum(container[hash_id]['history']) / len(container[hash_id]['history'])

                    # ERROR MARGIN
                    margin = avg * error_margin

                    # PRINT MSG
                    print('EVENT:\t\t\t', hash_id)
                    print('DELTA:\t\t\t', delta)
                    print('AVG:\t\t\t', avg)

                    # CHECK IF DELTA IS NOT WITHIN RANGE
                    if value >= avg - margin and value <= avg + margin:
                        print('SUCCESS:\t\tTRUE')

                    # OTHERWISE
                    else:
                        print('SUCCESS:\t\tFALSE')

                # REGARDLESS, ADD DISTANCE DELTA TO HISTORY
                index = container[hash_id]['next_index']
                container[hash_id]['history'][index] = delta

                # SELECT THE NEXT HISTORY INDEX OR DEFUALT TO ZERO IF LIMIT IS EXCEEDED
                find_next = index + 1 if index + 1 <= history_length else 0

                # SET NEXT INDEX IN STATE
                container[hash_id]['next_index'] = find_next

            # OTHERWISE
            else:

                # CREATE AN ENTRY
                container[hash_id] = {
                    'history': []
                    'previous': timestamp,
                    'next_index': 0
                }

                # PRINT MSG
                print('NEW EVENT DETECTED!')
                print(hash_id)
                print('')

        # OTHERWISE, SLEEP FOR A SECOND TO PREVENT SPAM
        else:
            time.sleep(1)


# KILL THE PROCESS
except:
    print('\nTHE PROCESS WAS MANUALLY KILLED')

STARTED LISTENING...



IndexError: list assignment index out of range